

# **SPAM DETECTION**

---

---





important packages and constants

In [2]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
# specify location your dataset here
DATA_PATH = "data/smsdb.txt"

# give name to label-column and text-column
COLUMN_LABEL = "label"
COLUMN_TEXT = "text"

# these are labels that indicate the type of message.
LABEL_LEGIT = 'LEGI'
LABEL_SPAM = 'SPAM'
LABEL_SMISHING = 'SMIS'

# **Dataset**

In [4]:
dataset = pd.read_csv(DATA_PATH, sep='\t', names=[COLUMN_LABEL, COLUMN_TEXT], header=None)
print('Total size:', dataset.shape[0])
print('Legit messages:', dataset[dataset[COLUMN_LABEL] == LABEL_LEGIT].shape[0])
print('Spam messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SPAM].shape[0])
print('Smishing messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SMISHING].shape[0])

Total size: 1533
Legit messages: 1055
Spam messages: 176
Smishing messages: 302


In [5]:
dataset = dataset[((dataset[COLUMN_LABEL] == LABEL_LEGIT) | (dataset[COLUMN_LABEL] == LABEL_SPAM))]

# Let's check if they are gone
print('Smishing messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SMISHING].shape[0])
print(len(dataset))

Smishing messages: 0
1231


# Data preprocessing

In [6]:
def messages2vectors(messages):
    '''
    Transforms single message into feature-vector;
    Parameters:
        messages    -   array of strings;
    Returns:
        features    -   array of feature-vectors;   
    '''

    elmo = hub.Module("https://tfhub.dev/google/elmo/1")

    features = np.zeros((0, 1024))
    n = 100
    l = int(len(messages) / n) if len(messages) % n == 0 else int(len(messages) / n) + 1
    for i in range(l):

        if (i + 1) * n < len(messages):
            right = (i + 1) * n
            embedds = elmo(messages[int(i * n) : right], signature="default", as_dict=True)["default"] 
        else:
            embedds = elmo(messages[:len(messages) - int(i * n)], signature="default", as_dict=True)["default"] 

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            embedds = sess.run(embedds)
            features = np.concatenate([features, embedds])

    return features

In [7]:
#print(features)

In [8]:
def convert_labels(labels_raw):
    '''
    Transforms labels into numerical values;
    Parameters:
        labels_raw    -   array of text-labels;
    Returns:
        features    -   array of numerical labels;   
    ''' 

    # add your code here
    labels = []

    for i in labels_raw:
      if i == "LEGI":
        labels.append(0)
      elif i == "SPAM":
        labels.append(1)
      else:
        pass
    labels = np.asarray(labels)
    return labels

In [9]:
#print(labels)
#print(len(labels))

In [10]:
features = messages2vectors(dataset[COLUMN_TEXT])
labels = convert_labels(dataset[COLUMN_LABEL])
print(features.shape)
print(labels.shape)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1231, 1024)
(1231,)


In [11]:
def split_data(features, labels, ratio=0.7):
    '''
    Splits dataset into train/test parts using given ratio;
    Parameters:
        data    -   array of features;
        labels  -   array of corresponding labels;
        ratio   -   train/test size ratio;
    Returns:
        train_data      -   array of training features;   
        train_labels    -   array of training labels; 
        test_data       -   array of testing features; 
        test_labels     -   array of testing labels; 
    '''    


    positive_data = features[labels == 1] # all spam features
    negative_data = features[labels == 0] # all legit features

    # We shuffle arrays to get random samples later
    random_indecies_positive = np.arange(positive_data.shape[0])
    np.random.shuffle(random_indecies_positive)
    random_indecies_negative = np.arange(negative_data.shape[0])
    np.random.shuffle(random_indecies_negative)

    n_positive_train = int(positive_data.shape[0] * ratio)
    n_negative_train = int(negative_data.shape[0] * ratio)

    # Training data are all indecies in 'ratio' part of shuffled indecies
    train_data = np.concatenate([positive_data[random_indecies_positive[:n_positive_train]], 
                                negative_data[random_indecies_negative[:n_negative_train]]])
    
    train_labels = np.asarray([1] * n_positive_train + [0] * n_negative_train)

    # Testing data are all indecies that remain
    test_data = np.concatenate([positive_data[random_indecies_positive[n_positive_train:]], 
                                negative_data[random_indecies_negative[n_negative_train:]]])

    test_labels = np.asarray([1] * (positive_data.shape[0]  - n_positive_train) + [0] * (negative_data.shape[0] - n_negative_train))

    return train_data, train_labels, test_data, test_labels

# Metrics

In [1]:

def get_metrics(labels, predictions):
    '''
    Computes metrics;
    Parameters:
        labels    -   array of labels;
        predictions  -   array of predictions;
    Returns:
        FAR -   False Acceptance Rate;
        FRR -   False Rejection Rate;
        FAR = FPR = FP/(FP + TN)
        FRR = FNR = FN/(FN + TP)
        where FP: False positive
              FN: False Negative
              TN: True Negative
              TP: True Positive
              TP = cm[0][0]
              FP = cm[0][1]
              FN = cm[1][0]
              TN = cm[1][1]
         
    '''  
    
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(labels, predictions).ravel()
    print("tn, fp, fn, tp", tn, fp, fn, tp)

    # >FAR like FNR

    FRR = fp / (fp + tn) * 100
    FAR = fn / (fn + tp) * 100
#
    #print('FAR, FRR',FAR, FRR)

    #FAR, tpr, thresholds = sklearn.metrics.roc_curve(labels, predictions)
    #FRR = 1-tpr
    #print('FAR, FRR',FAR, FRR)
    #
    #FRR = (2 * sklearn.metrics.balanced_accuracy_score(labels, predictions)) - sklearn.metrics.recall_score(labels, predictions)
    #FAR = 1 - sklearn.metrics.recall_score(labels, predictions)
    
    return FAR, FRR

#Model initialization

In [12]:
classifierType = sklearn.ensemble.RandomForestClassifier
hyperparameters = {'n_estimators' : 100,
                'criterion' : 'gini',
                'max_depth' : None,
                'min_samples_split' : 2}

# Model Training and evaluation

In [13]:
def evaluate(classifierType, hyperparameters, features, labels):
    '''
    Splits dataset into train/test parts using given ratio;
    Parameters:
        classifierType      -   type of ML algorithm to use;
        hyperparameters     -   dictionary of model's parameters;
        features            -   array of features;
        labels              -   array of labels
    Returns:
        trainFAR    -   False Acceptance Rate for train dataset;
        trainFRR    -   False Rejection Rate for train dataset;
        testFAR     -   False Acceptance Rate for test dataset;
        testFRR    -   False Rejection Rate for test dataset;

        train_data      -   array of training features;   
        train_labels    -   array of training labels; 
        test_data       -   array of testing features; 
        test_labels     -   array of testing labels; 
        FAR = FPR = FP/(FP + TN)
        FRR = FNR = FN/(FN + TP)
        where FP: False positive
              FN: False Negative
              TN: True Negative
              TP: True Positive
    
    '''    
    
    model = classifierType(**hyperparameters)

    # Split data
    # add your code here
    train_data, test_data, train_labels, test_labels = train_test_split(features, labels)
    #X_train, X_test, y_train, y_test train_data, train_labels, test_data, test_labels
    print('Train set shape:', train_data.shape)
    print('Train labels shape:', train_labels.shape)
    print('Test set shape:', test_data.shape)
    print('Test labels shape:', test_labels.shape)

    #print("train data", train_data)
    #print("train labels", train_labels)
    #print("test data", test_data) #binary
    #print("test labels", test_labels) #binary

    # Fit your model
    # add your code here
    model.fit(train_data, train_labels)

    # Make predictions for training dataset
    # add your code here
    pred = model.predict(train_data)
    print('predict', pred.shape)
    # Compute train FAR/FRR
    # add your code here
    trainFAR, trainFRR = get_metrics(train_labels, pred)
    print('trainFAR, trainFRR',trainFAR, trainFRR)

    # Make predictions for testing dataset
    # add your code here
    predictions_test = model.predict(test_data)

    # Compute test FAR/FRR
    # add your code here
    testFAR, testFRR =get_metrics(test_labels, predictions_test)
    print('testFAR, testFRR',testFAR, testFRR)


    return trainFAR, trainFRR, testFAR, testFRR

In [14]:
# Check if it works :)
trainFAR, trainFRR, testFAR, testFRR = evaluate(classifierType, hyperparameters, features, labels)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train set shape: (923, 1024)
Train labels shape: (923,)
Test set shape: (308, 1024)
Test labels shape: (308,)
predict (923,)
tn, fp, fn, tp 782 0 0 141
trainFAR, trainFRR 0.0 0.0
tn, fp, fn, tp 268 5 8 27
testFAR, testFRR 22.857142857142858 1.8315018315018317
Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 22.857142857142858
	FRR: 1.8315018315018317


# Final Task

 **SVM**

In [15]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels)
#X_train, X_test, y_train, y_test

In [16]:
from sklearn import svm

In [17]:
clf = svm.SVC(kernel='linear')
clf.fit(train_data, train_labels)
test_labels_pred = clf.predict(test_data)
train_labels_pred = clf.predict(train_data)

In [18]:
print("Accuracy test", sklearn.metrics.accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", sklearn.metrics.accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9805194805194806
Accuracy train 0.9978331527627302


In [19]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)

tn, fp, fn, tp 796 2 0 125
tn, fp, fn, tp 255 2 4 47


In [20]:
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.2506265664160401
Test:
	FAR: 7.8431372549019605
	FRR: 0.7782101167315175


**K-Nearest Neighbors**

In [21]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels)
#X_train, X_test, y_train, y_test

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_data, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [24]:
test_labels_pred = knn.predict(test_data)
train_labels_pred = knn.predict(train_data)

In [25]:
print("Accuracy test", sklearn.metrics.accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", sklearn.metrics.accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9642857142857143
Accuracy train 0.9772481040086674


In [26]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

tn, fp, fn, tp 771 18 3 131
tn, fp, fn, tp 256 10 1 41
Train:
	FAR: 2.2388059701492535
	FRR: 2.2813688212927756
Test:
	FAR: 2.380952380952381
	FRR: 3.7593984962406015


**Logistic Regression**

In [27]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels)
#X_train, X_test, y_train, y_test

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
lg = LogisticRegression()

In [30]:
lg.fit(train_data, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
test_labels_pred = lg.predict(test_data)
train_labels_pred = lg.predict(train_data)

In [32]:
print("Accuracy test", sklearn.metrics.accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", sklearn.metrics.accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9837662337662337
Accuracy train 0.9945828819068255


In [33]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

tn, fp, fn, tp 782 3 2 136
tn, fp, fn, tp 269 1 4 34
Train:
	FAR: 1.4492753623188406
	FRR: 0.3821656050955414
Test:
	FAR: 10.526315789473683
	FRR: 0.3703703703703704
